In [ ]:
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import Adam
#from generator import DataGenerator
import keras
from keras.callbacks import ModelCheckpoint
import os

In [ ]:
labels = os.listdir('../input/kermany2018/OCT2017 /train/')
train_datagen = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.05, 
                              width_shift_range=0.1, 
                              rotation_range=15, 
                              zoom_range=0.15)
validation_datagen = ImageDataGenerator(samplewise_center=True, 
                                        samplewise_std_normalization=True)
test_datagen = ImageDataGenerator(samplewise_center=True, 
                                  samplewise_std_normalization=True)

In [ ]:
IMG_SIZE = 224
batch_size = 16
train_data_dir = '../input/kermany2018/OCT2017 /train'
validation_data_dir = '../input/kermany2018/OCT2017 /val'
#test_data_dir = '../input/kermany2018/OCT2017 /test'
test_data_dir = '../input/mydata/myData'
valid_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(IMG_SIZE , IMG_SIZE),
    batch_size=32,
    #subset='validation',
    class_mode='categorical')
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_SIZE , IMG_SIZE),
    batch_size=16,
    #subset='training',
    class_mode='categorical',
    shuffle=True)

In [ ]:
t_x, t_y = next(train_generator)
fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone')
    c_ax.set_title(', '.join([n_class for n_class, n_score in zip(labels, c_y) 
                             if n_score>0.5]))
    c_ax.axis('off')

In [ ]:
# Defining ResNet model
t_x, t_y = next(train_generator)
def resnet_model():
    img_in = Input(t_x.shape[1:])              #input of model 
    model = ResNet50(include_top= False , # remove  the 3 fully-connected layers at the top of the network
                weights='imagenet',      # pre train weight 
                input_tensor= img_in, 
                input_shape= t_x.shape[1:],
                pooling ='avg') 
    x = model.output  
    predictions = Dense(4, activation="softmax", name="predictions")(x)    # fuly connected layer for predict class 
    model = Model(inputs=img_in, outputs=predictions)
    return model

In [ ]:

# compiling model
model = resnet_model()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                           metrics = ['accuracy'])


In [ ]:
# creating checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
# training model
history = model.fit_generator(train_generator, 
                                  steps_per_epoch=100,
                                  validation_data = valid_generator, 
                                  epochs = 20,
                                  callbacks=callbacks_list)

In [ ]:
print(history.history['accuracy'])

In [ ]:
# Accuracy plots
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# loading the model with best weight
filepath = "../input/weights23/ResNETLimitedweights.best.hdf5"
model = resnet_model()
model.load_weights(filepath)

In [ ]:
# creating test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_SIZE , IMG_SIZE),
    batch_size=24,
    class_mode='categorical')
test_X,test_Y = next(test_generator)

In [ ]:
import time
# predicting y value
start = time.time()
y_pred = model.predict(test_X)
end = time.time()
print('Test time per image:')
print((end - start)/968)

In [ ]:
import numpy as np

pred_class = []
for i in range(len(y_pred)):
    pred_class.append(np.argmax(y_pred[i]))



In [ ]:
actual_class = []
for i in range(len(test_Y)):
    actual_class.append(np.argmax(test_Y[i]))

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

print(pred_class)
print(actual_class)
print(classification_report(actual_class, pred_class))

In [ ]:
# accuracy score
from sklearn.metrics import accuracy_score
print('accuracy = ',accuracy_score(pred_class,actual_class))


In [ ]:
# defining confusion matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(test_y, predict_y):
    C = confusion_matrix(test_y, predict_y)
    # C = 9,9 matrix, each cell (i,j) represents number of points of class i are predicted class j
    
    A =(((C.T)/(C.sum(axis=1))).T)
    #divid each element of the confusion matrix with the sum of elements in that column
    
    # C = [[1, 2],
    #     [3, 4]]
    # C.T = [[1, 3],
    #        [2, 4]]
    # C.sum(axis = 1)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =1) = [[3, 7]]
    # ((C.T)/(C.sum(axis=1))) = [[1/3, 3/7]
    #                           [2/3, 4/7]]

    # ((C.T)/(C.sum(axis=1))).T = [[1/3, 2/3]
    #                           [3/7, 4/7]]
    # sum of row elements = 1
    
    B =(C/C.sum(axis=0))
    #divid each element of the confusion matrix with the sum of elements in that row
    # C = [[1, 2],
    #     [3, 4]]
    # C.sum(axis = 0)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =0) = [[4, 6]]
    # (C/C.sum(axis=0)) = [[1/4, 2/6],
    #                      [3/4, 4/6]] 
    
    labels = [0,1,2,3]
    # representing A in heatmap format
    print("-"*20, "Confusion matrix", "-"*20)
    plt.figure(figsize=(20,7))
    sns.heatmap(C, annot=True, cmap="YlGnBu", fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()

    print("-"*20, "Precision matrix (Columm Sum=1)", "-"*20)
    plt.figure(figsize=(20,7))
    sns.heatmap(B, annot=True, cmap="YlGnBu", fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    
    # representing B in heatmap format
    print("-"*20, "Recall matrix (Row sum=1)", "-"*20)
    plt.figure(figsize=(20,7))
    sns.heatmap(A, annot=True, cmap="YlGnBu", fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()

In [ ]:
plot_confusion_matrix(actual_class,pred_class)